# training 10 features to 4 classes
## one vs all

weights is size 10 (should be 11 ?)
replacing nan with 0

Removing


replacing nan with 0

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def standardize(arr:np.ndarray):
    mean = np.mean(arr)
    std = np.std(arr)
    return (arr - mean) / std

def sigmoid(arr:np.ndarray):
    return 1 / (1 + np.exp(-arr))

def loss_function(y_actual, h_pred):
    """ y_actual : target class. 1 in class, 0 not in class
    h_pred = signoid(x.weights)
    loss = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    """
    m = len(h_pred)
    a = -y_actual * np.log(h_pred)
    b = (1 - y_actual) * np.log(1 - h_pred)
    return (a - b) / m

def update_weight_loss(weights, learning_rate, grad_desc):
    return weights - learning_rate * grad_desc

In [63]:
df = pd.read_csv(f'../datasets/dataset_train.csv')
df.drop(df.columns[2:6], inplace=True, axis = 1)
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df.drop(excluded_features, inplace=True, axis=1)
df

,Index,Hogwarts House,Astronomy,Herbology,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Charms,Flying
0,0,Ravenclaw,-487.886086,5.727180,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,-232.79405,-26.89
1,1,Slytherin,-552.060507,-5.987446,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,-252.18425,-113.45
2,2,Ravenclaw,-366.076117,7.725017,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-227.34265,30.42
3,3,Gryffindor,697.742809,-6.497214,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-256.84675,200.64
4,4,Gryffindor,436.775204,-7.820623,2.236,-444.262537,599.324514,-3.444377,937.434724,4.311066,-256.38730,157.98
...,...,...,...,...,...,...,...,...,...,...,...,...
1595,1595,Gryffindor,354.280086,-4.541837,5.702,-497.235066,618.220213,-5.231721,964.219853,3.389086,-250.39401,185.83
1596,1596,Slytherin,367.531174,6.061064,1.757,-643.271092,445.827565,2.238112,1056.147366,5.825263,-246.42719,44.80
1597,1597,Gryffindor,544.018925,-3.203269,6.065,-385.150457,635.211486,-5.984257,953.866685,1.709808,-251.63679,198.47
1598,1598,Hufflepuff,453.676219,3.442831,6.738,-831.741123,383.444937,3.813111,1087.949205,3.904100,-246.19072,-76.81


### drop NaN
1333 from 1600
### add a column of ones at begin

In [133]:
df_train = df.dropna()
target = df_train.columns[1]
features = df_train.columns[2:]
classes = df_train[target]
houses = classes.unique().tolist()
df_x_train = df_train[df_train.columns[2:]]
# standardize
df_train_std = df_x_train.agg(lambda feature: standardize(feature))
x_train_std = np.array(df_train_std)
# add a column of ones at begin
ones = np.ones((len(x_train_std), 1), dtype=float)
x_train = np.concatenate((ones, x_train_std), axis=1)

In [134]:
#df_losses = pd.DataFrame(columns=houses)
#df_losses = {}
w_indexes = df_train.columns[2:].insert(0, ['Bias'])
df_weights = pd.DataFrame(columns=houses, index=w_indexes)
df_weights.fillna(0)

,Ravenclaw,Slytherin,Gryffindor,Hufflepuff
Bias,0,0,0,0
Astronomy,0,0,0,0
Herbology,0,0,0,0
Divination,0,0,0,0
Muggle Studies,0,0,0,0
Ancient Runes,0,0,0,0
History of Magic,0,0,0,0
Transfiguration,0,0,0,0
Potions,0,0,0,0
Charms,0,0,0,0


In [136]:
learning_rate = 0.1
epochs = 1000
for house in houses:
    y_actual = np.where(classes == house, 1, 0)  #horiz
    # print(y_actual[:20])
    # loss, weights = train_one_vs_all(house, y_actual)
    loss = []
    weights = np.ones(len(features) + 1).T
    for iter in range(epochs):
        z_output = np.dot(x_train, weights)
        h_pred = sigmoid(z_output) 
        loss_iter = loss_function(y_actual, h_pred)
        #loss.append(loss_iter)
        gradient = np.dot(x_train.T, (h_pred - y_actual))
        grad_desc = np.dot(x_train.T, (h_pred - y_actual)) / y_actual.shape[0]
        # nor ok : grad_desc = gradient_descent(x_train, h_pred, gradient)
        weights = update_weight_loss(weights, learning_rate, grad_desc)
    #df_losses[house] = loss
    df_weights[house] = weights
df_weights.head(12)

,Ravenclaw,Slytherin,Gryffindor,Hufflepuff
Bias,-2.063096,-3.078773,-2.551754,-1.838897
Astronomy,-0.624236,-0.670175,0.744075,2.292742
Herbology,0.251666,-1.008444,-1.105922,1.198677
Divination,-0.124341,-1.982947,0.207340,0.582539
Muggle Studies,1.658823,-0.162779,0.083572,-0.767586
Ancient Runes,1.302175,-0.275755,1.209117,-1.222594
History of Magic,0.456105,0.376232,-0.460339,1.317395
Transfiguration,0.740743,0.885948,-0.592045,1.134280
Potions,-0.037885,0.618717,0.033944,-0.453786
Charms,0.960410,-0.606031,-0.472412,-0.395965


array([0.9599455 , 0.98270273, 0.94256671, 0.93792777, 0.96540937,
       0.97552457, 0.98700615, 0.98913055, 1.01608296, 0.95249828,
       0.99182784])

In [ ]:


class LogRegTrain:
    def __init__(self, df_to_train, df_class):
        ''' standardize df to train'''
        df_std = df_to_train.agg(lambda feature: LogRegTrain.standardize(feature))
        self.df_std= df_std
        self.x_train = np.array(df_std.fillna(0)) 
        self.x_train = np.array(df_std.dropna())  
        self.features = df_std.columns.to_list()
        self.df_class = df_class
        self.houses = ['Ravenclaw', 'Slytherin', 'Griffondor', 'Hufflepuff']
        #self.houses = df_class['Hogwarts House'].unique()
        self.df_losses = pd.DataFrame(columns=self.houses)
        self.df_losses.fillna(0)
        self.df_weights = pd.DataFrame(columns=self.houses, index=self.features)
        self.df_weights.fillna(0)

    def train_one_vs_all(self, house, y_actual):
        loss = []
        weights = np.ones(len(self.features)).T
        for iter in range(self.epochs):
            z_output = np.dot(self.x_train, weights)
            h_pred = LogRegTrain.sigmoid(z_output) 
            loss_iter = LogRegTrain.loss_function(y_actual, h_pred)
            loss.append(loss_iter)
            gradient = np.dot(self.x_train.T, (h_pred - y_actual))
            grad_desc = LogRegTrain.gradient_descent(self.x_train, h_pred, gradient)
            weights = LogRegTrain.update_weight_loss(weights, self.learning_rate, grad_desc)
            return loss, weights

    def train(self, learning_rate=0.1, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        for house in self.houses:
            y_actual = np.where(self.df_class[1] == house, 1, 0)
            loss, weights = self.train_one_vs_all(house, y_actual)
            self.df_losses[house] = loss
            self.df_weights[house] = weights

    @staticmethod
    def standardize(arr:np.ndarray):
        mean = np.mean(arr)
        std = np.std(arr)
        return (arr - mean) / std
    
    @staticmethod
    def sigmoid(arr:np.ndarray):
        return 1 / (1 + np.exp(-arr))
    
    @staticmethod
    def loss_function(y_actual, h_pred):
        """ y_actual : target class. 1 in class, 0 not in class
        h_pred = signoid(x.weights)
        loss = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
        """
        m = len(h_pred)
        a = -y_actual * np.log(h_pred)
        b = (1 - y_actual) * np.log(1 - h_pred)
        return (a - b) / m
    
    @staticmethod
    def gradient_descent(x_train, h_pred, y_actual):
        return np.dot(x_train.T, (h_pred - y_actual)) / y_actual.shape[0]

    @staticmethod
    def update_weight_loss(weights, learning_rate, grad_desc):
        return weights - learning_rate * grad_desc

In [30]:

df = pd.read_csv(f'../datasets/dataset_train.csv')
df_class = df['Hogwarts House']
df_features = df[df.columns[6:]]
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df_features = df_features.drop(excluded_features, axis=1)
# unstandardized data to train
logreg_model = LogRegTrain(df_features, df_class)
logreg_model.train(0.1, 50)
sns.lineplot(data=logreg_model.df_losses, legend='auto')

ValueError: operands could not be broadcast together with shapes (1600,) (10,) 